# 1. Title & Objective

# Assignment: Conversation Management & Classification using Groq API

**Objective:**  
Implement two core tasks using Groq APIs (OpenAI-compatible) without frameworks:

1. **Task 1:** Manage conversation history with summarization and truncation.  
2. **Task 2:** Extract structured information from chats using JSON schema and function calling.


# 2. Imports & Setup

In [6]:
import os
import time
import json
import textwrap
from getpass import getpass
from typing import List, Dict, Any, Optional
import requests

try:
    import jsonschema
except Exception:
    print("jsonschema not found. Installing...")
    !pip install jsonschema
    import jsonschema

print("--- Secure API key setup ---")
GROQ_API_KEY = getpass('Enter your GROQ API key (input hidden): ')
if not GROQ_API_KEY:
    raise ValueError("API key is required to run demo calls. Set GROQ_API_KEY as an environment variable or paste it when asked.")

os.environ['GROQ_API_KEY'] = GROQ_API_KEY



--- Secure API key setup ---
Enter your GROQ API key (input hidden): ··········


# 3. Groq API Wrapper

In [7]:
# --- Groq OpenAI-compatible API wrapper (copy-paste this entire cell) ---
import os, json, re, requests
from typing import List, Dict, Any, Optional

# Use the OpenAI-compatible Groq base path
GROQ_API_BASE = os.environ.get('GROQ_API_BASE', 'https://api.groq.com/openai/v1')
CHAT_COMPLETIONS_URL = f"{GROQ_API_BASE}/chat/completions"

# Default model: change this to any model id from your "Available models" list
DEFAULT_MODEL = "llama-3.3-70b-versatile"

def _realistic_mock_response(messages, functions):
    """
    Heuristic/mock fallback: attempts simple regex extraction so demos look realistic
    when a real API call fails or no key is provided.
    """
    full_text = " ".join([m.get('content','') for m in messages]).strip()
    name_match = re.search(r"(?:I am|I'm|Name:|this is)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)", full_text)
    email_match = re.search(r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})", full_text)
    phone_match = re.search(r"(\+?\d{1,3}[\s-]?\d{3,}[\d\s-]{3,})", full_text)
    location_match = re.search(r"(?:based in|Location:|in)\s+([A-Za-z][A-Za-z\s-]{1,50}?)(?:[.,;]|$)", full_text)
    age_match = re.search(r"(?:I am|I'm|age[:\s]*)\s*(\d{1,2})(?:\s*(?:years old|yrs|y/o))?", full_text, flags=re.IGNORECASE)

    fake_args = {}
    if name_match: fake_args['name'] = name_match.group(1).strip()
    if email_match: fake_args['email'] = email_match.group(1).strip()
    if phone_match: fake_args['phone'] = phone_match.group(1).strip()
    if location_match: fake_args['location'] = location_match.group(1).strip().rstrip('.')
    if age_match:
        try: fake_args['age'] = int(age_match.group(1))
        except: pass

    defaults = {"name":"Unknown","email":"not_provided@example.com","phone":"not_provided","location":"not_provided","age":None}
    for k,v in defaults.items(): fake_args.setdefault(k,v)

    if functions:
        return {"choices":[{"message":{"function_call":{"name": functions[0].get('name','extract_user_info'), "arguments": json.dumps(fake_args)}}}]}
    else:
        return {"choices":[{"message":{"content":"MOCK SUMMARY: (demo) summary placeholder."}}]}

def groq_chat_completion(messages: List[Dict[str, str]],
                         functions: Optional[List[Dict[str, Any]]] = None,
                         function_call: Optional[Any] = 'auto',
                         model: str = DEFAULT_MODEL,
                         max_tokens: int = 800,
                         timeout: int = 15) -> Dict[str, Any]:
    """
    Robust Groq/OpenAI-compatible chat wrapper.

    - Uses /openai/v1 path (correct for Groq's OpenAI-compatible API).
    - If GROQ_API_KEY present in environment, attempts real API call.
    - On HTTP error or other failures, prints diagnostics and falls back to a realistic mock response.
    - `functions` and `function_call` are passed through (for function-calling flows).
    """
    payload = {'model': model, 'messages': messages, 'max_tokens': max_tokens}
    if functions is not None:
        payload['functions'] = functions
        payload['function_call'] = function_call

    key = os.environ.get('GROQ_API_KEY','')
    if not key:
        print("⚠️ No GROQ_API_KEY found in environment — running MOCK mode.")
        return _realistic_mock_response(messages, functions)

    headers = {'Authorization': f"Bearer {key}", 'Content-Type': 'application/json'}
    try:
        resp = requests.post(CHAT_COMPLETIONS_URL, headers=headers, json=payload, timeout=timeout)
        resp.raise_for_status()
        return resp.json()
    except requests.HTTPError as http_err:
        code = getattr(http_err.response,'status_code',None)
        print(f"⚠️ HTTP error calling Groq API: {http_err} (status {code})")
        try:
            body = http_err.response.text
            snippet = (body[:1000] + "... [truncated]") if len(body) > 1000 else body
            print("Server response (truncated):", snippet)
        except Exception:
            pass
        print("Falling back to realistic MOCK response so the notebook can continue.")
        return _realistic_mock_response(messages, functions)
    except Exception as e:
        print("⚠️ Error calling Groq API:", e)
        print("Falling back to realistic MOCK response so the notebook can continue.")
        return _realistic_mock_response(messages, functions)

# Quick self-check (optional) — run this cell and examine printed output
if __name__ == "__main__":
    _test_msgs = [{'role':'user','content':"Self-check: say hello."}]
    print("Wrapper ready. Self-check result (may be mock if key/model not available):")
    print(groq_chat_completion(_test_msgs))



Wrapper ready. Self-check result (may be mock if key/model not available):
{'id': 'chatcmpl-83188699-d6a0-44a6-a5e3-b21a4b1c86b1', 'object': 'chat.completion', 'created': 1757945914, 'model': 'llama-3.3-70b-versatile', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hello.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'queue_time': 0.04630326, 'prompt_tokens': 41, 'prompt_time': 0.01100546, 'completion_tokens': 3, 'completion_time': 2.4e-07, 'total_tokens': 44, 'total_time': 0.0110057}, 'usage_breakdown': None, 'system_fingerprint': 'fp_2ddfbb0da0', 'x_groq': {'id': 'req_01k56vvrq8e97t8jqqk777qwsg'}, 'service_tier': 'on_demand'}


# 4. ConversationManager Class

In [8]:
# --- Updated ConversationManager (use the same DEFAULT_MODEL as wrapper) ---
class ConversationManager:
    def __init__(self, periodic_k: int = 3, summarization_model: str = DEFAULT_MODEL):
        """
        periodic_k: perform summarization after every k runs
        summarization_model: model id to use for summarization (defaults to DEFAULT_MODEL)
        """
        self.history = []  # list of {'role':..., 'content':...}
        self.run_count = 0
        self.periodic_k = periodic_k
        self.summarization_model = summarization_model

    def add_message(self, role: str, content: str):
        self.history.append({'role': role, 'content': content})

    def last_n_turns(self, n: int):
        return self.history[-n:]

    def truncate_by_chars(self, max_chars: int):
        out = []
        total = 0
        for msg in reversed(self.history):
            msg_len = len(msg['content'])
            if total + msg_len > max_chars:
                break
            out.insert(0, msg)
            total += msg_len
        return out

    def summarize_history(self, prompt_extra: Optional[str] = None) -> str:
        history_text = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.history])
        system_msg = {
            'role': 'system',
            'content': 'You are a helpful summarizer. Produce a short, bullet or paragraph summary of the conversation focusing on facts and user needs.'
        }
        user_msg = {
            'role': 'user',
            'content': 'Summarize the conversation below into 2-4 short bullet points.\n\nConversation:\n' + history_text + (('\n\n' + prompt_extra) if prompt_extra else '')
        }
        # explicitly pass model so it doesn't use an outdated default
        resp = groq_chat_completion([system_msg, user_msg], model=self.summarization_model, max_tokens=300)
        try:
            return resp['choices'][0]['message']['content']
        except Exception:
            return str(resp)

    def maybe_periodic_summarize(self):
        self.run_count += 1
        print(f"Run count: {self.run_count} (periodic_k={self.periodic_k})")
        if self.periodic_k > 0 and self.run_count % self.periodic_k == 0:
            print("Performing periodic summarization...")
            summary = self.summarize_history()
            self.history = [{'role': 'system', 'content': 'Summary of previous conversation: ' + summary}]
            return summary
        return None



# 5. Task 1

In [12]:
def demo_task1():
    cm = ConversationManager(periodic_k=3)
    samples = [
        ("user", "Hi, I'm Sanika. I'm looking for internship advice."),
        ("assistant", "Sure — what field are you interested in?"),
        ("user", "AI/ML roles, but I have more Python than ML projects."),
        ("assistant", "You can start with small projects: ..."),
        ("user", "Also I want to improve my resume formatting."),
        ("assistant", "Add project bullets, quantify impact, use clear headers."),
    ]

    print('\nFeeding samples and showing truncation settings...')
    for role, text in samples:
        cm.add_message(role, text)

    print('\n-- Last 3 turns --')
    for m in cm.last_n_turns(3):
        print(f"{m['role']}: {m['content']}")

    print('\n-- Truncate by 120 chars --')
    truncated = cm.truncate_by_chars(120)
    for m in truncated:
        print(f"{m['role']}: {m['content']}")

    print('\n-- Demonstrate periodic summarization after every 3 runs --')
    for i in range(1, 7):
        cm.add_message('user', f'Follow-up message #{i}')
        summary = cm.maybe_periodic_summarize()
        if summary:
            print('Summary created:')
            print(summary)
            print('History now contains:')
            print(cm.history)

demo_task1()



Feeding samples and showing truncation settings...

-- Last 3 turns --
assistant: You can start with small projects: ...
user: Also I want to improve my resume formatting.
assistant: Add project bullets, quantify impact, use clear headers.

-- Truncate by 120 chars --
user: Also I want to improve my resume formatting.
assistant: Add project bullets, quantify impact, use clear headers.

-- Demonstrate periodic summarization after every 3 runs --
Run count: 1 (periodic_k=3)
Run count: 2 (periodic_k=3)
Run count: 3 (periodic_k=3)
Performing periodic summarization...
Summary created:
Here is a summary of the conversation in 3 bullet points:

* Sanika is looking for internship advice, specifically in AI/ML roles.
* Sanika has more Python projects than ML projects, and may need to start with small projects to build experience.
* Sanika also wants to improve their resume formatting, with suggestions including adding project bullets, quantifying impact, and using clear headers.
History now co

# 6. Task 2: JSON Schema Extraction

In [10]:
FUNCTION_SCHEMA = {
    'name': 'extract_user_info',
    'description': 'Extract user contact and personal info from a chat message',
    'parameters': {
        'type': 'object',
        'properties': {
            'name': {'type': 'string'},
            'email': {'type': 'string'},
            'phone': {'type': 'string'},
            'location': {'type': 'string'},
            'age': {'type': 'integer'}
        },
        'required': []
    }
}

def call_extraction_function(chat_text):
    system_msg = {'role': 'system', 'content': 'You are a precise extractor. Return only JSON following the schema.'}
    user_msg = {'role': 'user', 'content': 'Extract contact info: ' + chat_text}
    resp = groq_chat_completion([system_msg, user_msg], functions=[FUNCTION_SCHEMA], function_call={'name': 'extract_user_info'})
    fc = resp['choices'][0]['message'].get('function_call')
    if fc and 'arguments' in fc:
        return json.loads(fc['arguments'])
    return {}

SAMPLE_CHATS = [
    "Hey I'm Priya Sharma. You can reach me at priya.sharma@example.com or call me on +91 98765 43210. I'm based in Pune and I'm 24.",
    "Hello, this is Amit. Email: amit123@mail.com. Location: Mumbai. Phone: 022-555-1234.",
    "Hi, I am 30 and live in Bangalore. Contact me at 9876543210. Name: Rahul."
]

for chat in SAMPLE_CHATS:
    print("\nChat:", chat)
    print("Extracted:", call_extraction_function(chat))



Chat: Hey I'm Priya Sharma. You can reach me at priya.sharma@example.com or call me on +91 98765 43210. I'm based in Pune and I'm 24.
Extracted: {'age': 24, 'email': 'priya.sharma@example.com', 'location': 'Pune', 'name': 'Priya Sharma', 'phone': '+91 98765 43210'}

Chat: Hello, this is Amit. Email: amit123@mail.com. Location: Mumbai. Phone: 022-555-1234.
Extracted: {'email': 'amit123@mail.com', 'location': 'Mumbai', 'name': 'Amit', 'phone': '022-555-1234'}

Chat: Hi, I am 30 and live in Bangalore. Contact me at 9876543210. Name: Rahul.
Extracted: {'age': 30, 'location': 'Bangalore', 'name': 'Rahul', 'phone': '9876543210'}
